In [2]:
# XGBoost on Otto dataset, Tune n_estimators
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier 


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import accuracy_score

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

import os
import time
from datetime  import datetime

In [3]:
strategyName='Long'

pahtname=r'D:\DataSets\ML_LS\ML-Long_S50M15_07To2Q20-Train_Norm.csv'

filepath= os.path.abspath(pahtname)

dataset =pd.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
dataset.drop(columns=['open','high','low','close'],inplace=True)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70896 entries, 2007-01-03 09:45:00 to 2020-06-30 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             70896 non-null  float64
 1   indy_ma-1100            70896 non-null  float64
 2   indy_hh-550             70896 non-null  float64
 3   indy_ll-550             70896 non-null  float64
 4   indy_mid-550            70896 non-null  float64
 5   indy_hh2-1100           70896 non-null  float64
 6   indy_ll2-1100           70896 non-null  float64
 7   indy_mid2-1100          70896 non-null  float64
 8   indy_macd110-440        70896 non-null  float64
 9   indy_signal110-440-110  70896 non-null  float64
 10  indy_hist_macd110-440   70896 non-null  float64
 11  indy_rsi25-ma20         70896 non-null  float64
 12  indy_6ATRTrail_DC-110   70896 non-null  float64
 13  cate_3trend-550_ma110   70896 non-null  int64  
 14  cat

In [4]:
def CreateData(dfx):
 labelCol=dfx.columns[len(dfx.columns)-1]
 
 X_df_temp = dfx.drop(columns=labelCol)
 Y_df_temp = dfx[labelCol]

 Xtemp=X_df_temp.to_numpy()
 ytemp=Y_df_temp.to_numpy()

 return X_df_temp,Y_df_temp, Xtemp,ytemp

In [5]:
#My Train
st='01-2007'
ed='12-2020'
df_train=dataset[st:ed]
#df_train.info()
X_df,Y_df,X,y=CreateData( df_train)

print(X_df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70896 entries, 2007-01-03 09:45:00 to 2020-06-30 16:45:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             70896 non-null  float64
 1   indy_ma-1100            70896 non-null  float64
 2   indy_hh-550             70896 non-null  float64
 3   indy_ll-550             70896 non-null  float64
 4   indy_mid-550            70896 non-null  float64
 5   indy_hh2-1100           70896 non-null  float64
 6   indy_ll2-1100           70896 non-null  float64
 7   indy_mid2-1100          70896 non-null  float64
 8   indy_macd110-440        70896 non-null  float64
 9   indy_signal110-440-110  70896 non-null  float64
 10  indy_hist_macd110-440   70896 non-null  float64
 11  indy_rsi25-ma20         70896 non-null  float64
 12  indy_6ATRTrail_DC-110   70896 non-null  float64
 13  cate_3trend-550_ma110   70896 non-null  int64  
 14  cat

In [6]:
X_df.tail(5)

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2020-06-30 15:45:00,1.03,1.0,1.11,0.97,0.0,1.11,0.90,0.0,-33.34,-22.94,-10.41,57.53,1.00,2,1,4,3
2020-06-30 16:00:00,1.03,1.0,1.11,0.97,0.0,1.11,0.90,0.0,-33.44,-23.14,-10.30,57.88,1.00,2,1,4,3
2020-06-30 16:15:00,1.03,1.0,1.11,0.97,0.0,1.11,0.90,0.0,-33.54,-23.34,-10.20,57.93,1.00,2,1,4,3
2020-06-30 16:30:00,1.04,1.0,1.11,0.98,0.0,1.11,0.91,0.0,-33.66,-23.54,-10.12,57.28,1.01,2,1,4,3
2020-06-30 16:45:00,1.04,1.0,1.12,0.98,0.0,1.12,0.91,0.0,-33.78,-23.74,-10.04,56.61,1.01,2,1,4,3


In [7]:
Y_df.tail(5)

datetime
2020-06-30 15:45:00    0
2020-06-30 16:00:00    0
2020-06-30 16:15:00    0
2020-06-30 16:30:00    0
2020-06-30 16:45:00    0
Name: LongSignal, dtype: int64

In [8]:
Y_df.value_counts()

0    41328
1    29568
Name: LongSignal, dtype: int64

In [9]:
option=1

split_outer=7
split_inner=5

# set shuffle is false owning to time series data
isShuffle=False  # if True  set random_state=xseed else None
if isShuffle==False:
    xseed=None
else:
    xseed=1990


if option==1 :
 cv_outer =  TimeSeriesSplit(n_splits=split_outer,max_train_size=None)
elif option==2:
 cv_outer = KFold(n_splits=split_outer, shuffle=isShuffle, random_state=xseed) 
else:
 cv_outer = StratifiedKFold(n_splits=split_outer, shuffle=isShuffle, random_state=xseed)

outer_results = list()

In [10]:
t_Start=time.time()
print(datetime.now())

2020-08-05 22:02:03.621776


In [ ]:
# StratifiedKFold option=3
#for train_ix, test_ix in cv_outer.split(X,y):
for train_ix, test_ix in cv_outer.split(X):
   # split data
   #print('train:',len(train_ix))
   #print('test:',len(test_ix))
   X_train, X_test = X[train_ix, :], X[test_ix, :]
   y_train, y_test = y[train_ix], y[test_ix]

   # configure the cross-validation procedure
   if option==1 :
    cv_inner =  TimeSeriesSplit(n_splits=split_inner,max_train_size=None)
   elif option==2 :
    cv_inner=KFold(n_splits=split_inner, shuffle=isShuffle, random_state=xseed)
   else:
    cv_inner = StratifiedKFold(n_splits=split_inner, shuffle=isShuffle, random_state=xseed)
    
    

   

   # ===================tuning==============================
   scoreX='accuracy'
   xDepth=8
   #estimatorList = [50,100,150]
   estimatorList=range(40, 240,20)

   #learning_rateList =[0.1,0.5,1]
   learning_rateList =[0.001,0.005,0.01,0.05,0.1,0.5,1]

   param_grid = dict(learning_rate=learning_rateList, n_estimators=estimatorList)
   model = XGBClassifier(max_depth=xDepth,importance_type='gain')

   # define search
   search = GridSearchCV(model, param_grid, scoring=scoreX, cv=cv_inner, refit=True)
    
   # execute search
   result = search.fit(X_train, y_train)
    
   # get the best performing model fit on the whole training set
   best_model = result.best_estimator_
    
    # evaluate model on the hold out dataset
   yhat = best_model.predict(X_test) 
    
    # evaluate the model
   acc = accuracy_score(y_test, yhat)

    # store the result
   outer_results.append(acc)
    
    # report progress
   print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))


>acc=0.853, est=0.918, cfg={'learning_rate': 1, 'n_estimators': 40}
>acc=0.862, est=0.898, cfg={'learning_rate': 0.05, 'n_estimators': 140}
>acc=0.824, est=0.876, cfg={'learning_rate': 0.05, 'n_estimators': 160}
>acc=0.939, est=0.866, cfg={'learning_rate': 0.1, 'n_estimators': 60}
>acc=0.834, est=0.868, cfg={'learning_rate': 0.5, 'n_estimators': 220}


In [ ]:
print(datetime.now())
t_End=time.time()
t_elapsed=(t_End-t_Start)/60/60
print('Total execute train : ',round(t_elapsed,2))

In [ ]:
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))